# Q-Learning in the GridWorld environment

### Loading Libraries

In [31]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
from mdptoolbox import mdp
from itertools import product

# Time & Path
import time
from pathlib import Path
from time import process_time

In [32]:
%matplotlib inline

### Set Gridworld Up 

#### States, Actions & Rewards

In [36]:
grid_size = (3, 4)

blocked_cell = (1, 1)

baseline_reward = -0.02

absorbing_cells = {(0, 3): 1, (1, 3): -1}

In [38]:
actions = ['L', 'U', 'R', 'D']

num_actions = len(actions)

probs = [.1, .8, .1, 0]

In [40]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)

to_2d = lambda x: np.unravel_index(x, grid_size)

In [42]:
num_states = np.product(grid_size)

cells = list(np.ndindex(grid_size))

states = list(range(len(cells)))

In [44]:
cell_state = dict(zip(cells, states))

state_cell= dict(zip(states, cells))

In [46]:
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}

blocked_state = to_1d(blocked_cell)

In [48]:
state_rewards = np.full(num_states, baseline_reward)

state_rewards[blocked_state] = 0

for state, reward in absorbing_states.items():
    state_rewards[state] = reward

In [50]:
action_outcomes = {}

for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

In [52]:
action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

#### Transition Matrix

In [55]:
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

In [57]:
state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

In [59]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

In [61]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))
transitions = np.zeros((num_actions, num_states, num_states))
actions_ = list(range(num_actions))
for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

In [63]:
rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

### Q-Learning

In [66]:
max_episodes = 2500

alpha = .1

epsilon = .05

gamma = .99

In [68]:
Q = np.random.rand(num_states, num_actions)

skip_states = list(absorbing_states.keys())+[blocked_state]

Q[skip_states] = 0

In [70]:
start = process_time()

for episode in range(max_episodes):
    state = np.random.choice([s for s in states if s not in skip_states])
    while not state in absorbing_states.keys():
        if np.random.rand() < epsilon:
            action = np.random.choice(num_actions)
        else:
            action = np.argmax(Q[state])
        next_state = np.random.choice(states, p=transitions[action, state])
        reward = rewards[action, state, next_state]
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state])-Q[state, action])
        state = next_state
process_time() - start

0.1952569999999998

In [72]:
pd.DataFrame(np.argmax(Q, 1).reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,D


In [74]:
pd.DataFrame(np.max(Q, 1).reshape(grid_size))

,0,1,2,3
0,0.871176,0.910151,0.949679,0.000000
1,0.830930,0.000000,0.698737,0.000000
2,0.795885,0.753633,0.723461,0.558584


### PyMDPToolbox

#### Q Learning

In [77]:
start = process_time()

ql = mdp.QLearning(transitions=transitions,
                   reward=rewards,
                   discount=gamma,
                   n_iter=int(1e6))

ql.run()
f'Time: {process_time()-start:.4f}'

'Time: 3.3464'

In [78]:
policy = np.asarray([actions[i] for i in ql.policy])

pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,U,R,U,L


In [79]:
value = np.asarray(ql.V).reshape(grid_size)

pd.DataFrame(value)

,0,1,2,3
0,0.822882,0.922114,0.959083,0.00000
1,0.659741,0.000000,0.681807,0.00000
2,0.277596,0.409701,0.573479,0.26988
